# Manipulating Derived Attributes

This notebook works through loading data from prime attributes (those with literal values in the serialization) and derived attributes (those that are derived through reference to others). Because SysML v2 makes derived attributes come from the Relationships between non-Relationship elements, the Relationships will be examined first.

In [ ]:
import json
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Any, Dict, List, Set, Tuple, Union
from uuid import uuid4
from warnings import warn

## Important Infrastructure

This section covers major pieces of coding infrastructure that will be used in the rest of the methods in this notebook.

### Referencing Items by Their Name Rather than Index

In [ ]:
class ListOfNamedItems(list):
    """A list that also can return items by their name."""
    
    # FIXME: Should really build/update the item map on entry modification rather than lookup
    
    # FIXME: figure out why __dir__ of returned objects think they are lists
    def __getitem__(self, key):
        item_map = {
            item.name: item
            for item in self
            if isinstance(item, Element) and hasattr(item, "name")
        }
        effective_item_map = {
            item.effectiveName: item
            for item in self
            if isinstance(item, Element) and hasattr(item, "effectiveName")
        }
        if key in item_map:
            if printing_level == "TRACE":
                print(f"Adding {item_map[key]} to list index.")
            return item_map[key]
        if key in effective_item_map:
            if printing_level == "TRACE":
                print(f"Adding {effective_item_map[key]} to list index.")
            return effective_item_map[key]
        if isinstance(key, int):
            return super().__getitem__(key)
        return None

### ID reference format

The serialized JSON files will present IDs as a single entry dictionary of the form {'@id': uuid}.

In [ ]:
def is_id_item(item):
    is_unit_item = isinstance(item, dict) and item['@id'] is not None and isinstance(item['@id'], str)
    if is_unit_item:
        return True
    is_list_item = False
    if isinstance(item, list):
        for actual_item in item:
            if isinstance(actual_item, dict) and actual_item['@id'] is not None and isinstance(actual_item['@id'], str):
                return True
        
    return False

## Model Element Lookup

Tracing relationships requires the ability to look up Elements from a master register.

### Model Register

This class provides the ability to look up the set of loaded elements. This is set up as a class to support large models in case not the entire list of items can be stored in memory.

In [ ]:
@dataclass
class ModelRegistry():
    
    _elements: Dict[str, "Element"] = field(default_factory=dict)
    _id_list: List[str] = field(default_factory=list)
    
    def get(self, ele_id: str):
        return self._elements[ele_id]
    
    def get_all_of_metatype(self, metatype: str):
        return [ele for ele in self._elements.values() if ele._metatype == metatype]
        
    def get_all_ids(self):
        return self._id_list
    
    def add(self, ele):
        self._id_list.append(ele._id)
        self._elements.update({ele._id: ele}) 

## Prime Attribute Loading

The following section goes over the primary attributes that are to be loaded from and saved back to serializations of non-relationship elements.

### Partial Class

The Element class below has just the properties needed for this section.

In [ ]:
@dataclass
class Element:
    """Partial class to cover needs for lazy loading and derived attribute calculation methods."""
    
    to_lazy_derive: bool = True
    
    """The list of relationships which have this Element as a source"""
    source_to_relationships: Set[str] = field(default_factory=set)
    """The list of relationships which have this Element as a target"""
    target_to_relationships: Set[str] = field(default_factory=set)

In [ ]:
def primary_fields():
    primary_fields = ("elementId", "name", "effectiveName", "body", "isAbstract", 
                      "isUnique", "isOrdered", "isComposite", "isEnd", "@type", "@id")
    
    return primary_fields

In [ ]:
def load_primary_attributes(element=None, data={}):
    
    for d_key, d_val in data.items():
        if d_key in primary_fields():
            if d_key == "@id":
                setattr(element, "_id", d_val)
            elif d_key == "@type":
                setattr(element, "_metatype", d_val)
            else:
                setattr(element, d_key, d_val)

In [ ]:
def is_relationship(data):
    return "relatedElement" in data

## Relationship Attribute Loading

Relationships can have reference attributes loaded, but only the most important ones. The rest can be derived or implied through the subsetting and redefinition rules.

In [ ]:
def relationship_fields():
    
    relationship_fields = ("ownedRelatedElement", "ownedMemberElement", "owningRelatedElement", "ownedMemberFeature", "type", "typedFeature")
    
    return relationship_fields

In [ ]:
def load_relationship_attributes(element=None, data={}):
    
    for d_key, d_val in data.items():
        if d_key in relationship_fields() and is_id_item(d_val):
            if isinstance(d_val, list):
                flat_list = []
                for d_ind_val in d_val:
                    flat_list.append(d_ind_val['@id'])
                setattr(element, d_key, flat_list)
            else:
                setattr(element, d_key, [d_val['@id']])

In [ ]:
def dereference_relationship_ends(rel=None, registry=None):
    
    for field in relationship_fields():
        if hasattr(rel, field):
            field_vals = getattr(rel, field)
            need_reference = False
            
            # By this point, the relationship ends should all be lists, even if of length 1
            if field_vals is not None and isinstance(field_vals, list) and isinstance(field_vals[0], str):
                field_val_eles = []
                for field_ind_val in field_vals:
                    field_val_ele = registry.get(field_ind_val)
                    field_val_eles.append(field_val_ele)
                need_reference = True
                
            if need_reference:
                setattr(rel, field, field_val_eles)

## Loading Test

These cells walk through testing against serialized model data from the Reference Implementation API.

In [ ]:
with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data = json.load(fp)
    
printing_level = "ERROR"

In [ ]:
test_registry = ModelRegistry()

# Pass 1 - get non-relationship element data
for ele_data in element_raw_data:
    new_ele = Element()
    load_primary_attributes(element=new_ele, data=ele_data)
    if is_relationship(ele_data):
        load_relationship_attributes(element=new_ele, data=ele_data)
    test_registry.add(new_ele)
        
# Pass 2 - get relationship element data and connect others
for ele_data in element_raw_data:
    if is_relationship(ele_data):
        for source_ele in ele_data['source']:
            test_registry.get(source_ele['@id']).source_to_relationships.add(ele_data['@id'])
        for target_ele in ele_data['target']:
            test_registry.get(target_ele['@id']).target_to_relationships.add(ele_data['@id'])
        rel = test_registry.get(ele_data['@id'])
        dereference_relationship_ends(rel=rel, registry=test_registry)

In [ ]:
trial_items = test_registry.get_all_ids()[0:10]
[test_registry.get(trial_item) for trial_item in trial_items]

In [ ]:
owning_memberships = test_registry.get_all_of_metatype("OwningMembership")

In [ ]:
owning_memberships[0].ownedMemberElement

## Derived Attribute Calculations

### Owning Relationships

ASSUMPTION: The relevant relationships are binary, one source, one target.

The base ends for ownership are embedded in the root Relationship. Each Relationship has multiple "ownedRelatedElement" and "owningRelatedElement" fields. In practice, there is one of each in Relationships that specialize for ownership (e.g., OwningMembership, FeatureMembership, EndFeatureMembership)

For all rel : Relationship

sourceElement.ownedElement += rel.ownedRelatedElement

targetElement.owner = rel.owningRelatedElement

### OwningMembership

The OwningMembership is binary, and has a source and target. The source is the owningRelatedElement, while the target is the ownedMemberElement.

Source implications:
- redefined by membershipOwningNamespace
- subsets membershipNamespace
- subsets owningRelatedElement, which has opposite ownedMembership

Target implications:
- redefined by memberElement, ownedMemberElement
- subsets ownedRelatedElement, which has opposite owningRelationship

In [ ]:
def source_side_resolve_owningmembership(registry=None, source_ele=None, rel=None):
    if hasattr(source_ele, "ownedElement"):
        source_ele.ownedElement.append(registry.get(rel.ownedRelatedElement[0]._id))
    else:
        setattr(source_ele, "ownedElement", ListOfNamedItems([registry.get(rel.ownedRelatedElement[0]._id)]))
    if hasattr(source_ele, "throughOwningMembership"):
        source_ele.throughOwningMembership.append(registry.get(rel.ownedRelatedElement[0]._id))
    else:
        setattr(source_ele, "throughOwningMembership", ListOfNamedItems([registry.get(rel.ownedRelatedElement[0]._id)]))

In [ ]:
def target_side_resolve_owningmembership(registry=None, target_ele=None, rel=None):
    setattr(target_ele, "owner", [registry.get(rel.owningRelatedElement[0]._id)])
    
    if hasattr(target_ele, "reverseOwningMembership"):
        target_ele.reverseOwningMembership.append(registry.get(rel.owningRelatedElement[0]._id))
    else:
        setattr(target_ele, "reverseOwningMembership", [registry.get(rel.owningRelatedElement[0]._id)])

In [ ]:
trial_items = test_registry.get_all_ids()
for trial_item in trial_items:
    for source_rel in test_registry.get(trial_item).source_to_relationships:
        if test_registry.get(source_rel)._metatype == "OwningMembership":
            print('Resolving owned element for ' + trial_item)
            source_side_resolve_owningmembership(registry=test_registry,
                                                 source_ele=test_registry.get(trial_item),
                                                 rel=test_registry.get(source_rel))

In [ ]:
test_registry.get('0ae12f1c-e915-40b7-90dd-af4a62bf9e81').ownedElement

In [ ]:
[owned_ele.name for owned_ele in test_registry.get('0ae12f1c-e915-40b7-90dd-af4a62bf9e81').ownedElement]

In [ ]:
test_registry.get('0ae12f1c-e915-40b7-90dd-af4a62bf9e81').ownedElement["Transfer Torque"]

In [ ]:
test_registry.get('0ae12f1c-e915-40b7-90dd-af4a62bf9e81').name

### FeaturingMembership

The FeaturingMembership is binary, and has a source and target. The source is the owningRelatedElement, while the target is the ownedMemberElement.

Source implications:
- redefined by membershipOwningNamespace
- subsets membershipNamespace
- subsets owningRelatedElement, which has opposite ownedMembership

Target implications:
- redefined by ownedMemberFeature
- subsets ownedRelatedElement, which has opposite owningRelationship

In [ ]:
def source_side_resolve_featuremembership(registry=None, source_ele=None, rel=None):
    if hasattr(source_ele, "ownedElement"):
        source_ele.ownedElement.append(registry.get(rel.ownedMemberFeature[0]._id))
    else:
        setattr(source_ele, "ownedElement", ListOfNamedItems([registry.get(rel.ownedMemberFeature[0]._id)]))
    if hasattr(source_ele, "feature"):
        source_ele.feature.append(registry.get(rel.ownedMemberFeature[0]._id))
    else:
        setattr(source_ele, "feature", ListOfNamedItems([registry.get(rel.ownedMemberFeature[0]._id)]))
    if hasattr(source_ele, "throughFeatureMembership"):
        source_ele.throughFeatureMembership.append(registry.get(rel.ownedMemberFeature[0]._id))
    else:
        setattr(source_ele, "throughFeatureMembership", ListOfNamedItems([registry.get(rel.ownedMemberFeature[0]._id)]))

In [ ]:
def target_side_resolve_featuremembership(registry=None, target_ele=None, rel=None):
    setattr(target_ele, "owner", [registry.get(rel.owningRelatedElement[0]._id)])
    if hasattr(target_ele, "reverseFeatureMembership"):
        target_ele.reverseOwningMembership.append(registry.get(rel.owningRelatedElement[0]._id))
    else:
        setattr(target_ele, "reverseFeatureMembership", [registry.get(rel.owningRelatedElement[0]._id)])

In [ ]:
feature_memberships = test_registry.get_all_of_metatype("FeatureMembership")

In [ ]:
feature_memberships[0].ownedMemberFeature[0]._id

### FeatureTyping

The FeatureTyping is binary, and has a source and target. The source is the type, while the target is the typedFeature.

Source implications:
- redefines specific

Target implications:
- redefines general

In [ ]:
def source_side_resolve_featuretyping(registry=None, source_ele=None, rel=None):
    if hasattr(source_ele, "type"):
        source_ele.ownedElement.append(registry.get(rel.type[0]._id))
    else:
        setattr(source_ele, "type", [registry.get(rel.type[0]._id)])
    
    if hasattr(source_ele, "throughFeatureTyping"):
        source_ele.throughFeatureTyping.append(registry.get(rel.type[0]._id))
    else:
        setattr(source_ele, "throughFeatureTyping", ListOfNamedItems([registry.get(rel.type[0]._id)]))

In [ ]:
def target_side_resolve_featuretyping(registry=None, target_ele=None, rel=None):
    if hasattr(target_ele, "typedFeature"):
        target_ele.typedFeature.append(registry.get(rel.typedFeature[0]._id))
    else:
        setattr(target_ele, "typedFeature", [registry.get(rel.typedFeature[0]._id)])
    
    if hasattr(target_ele, "reverseFeatureTyping"):
        target_ele.reverseFeatureTyping.append(registry.get(rel.typedFeature[0]._id))
    else:
        setattr(target_ele, "reverseFeatureTyping", [registry.get(rel.typedFeature[0]._id)])

## Enacting Lazy Evaluation

The methods below allow Elements to perform a lazy evaluation of their derived attributes with respect to the relationships to which they are connected.

In [ ]:
def evaluate_derived_properties(ele=None, registry=None):
    for source_rel_id in ele.source_to_relationships:
        source_rel = registry.get(source_rel_id)
        if source_rel._metatype == "OwningMembership":
            source_side_resolve_owningmembership(registry=registry, source_ele=ele, rel=source_rel)
        elif source_rel._metatype == "FeatureMembership":
            source_side_resolve_featuremembership(registry=registry, source_ele=ele, rel=source_rel)
        elif source_rel._metatype == "FeatureTyping":
            source_side_resolve_featuretyping(registry=registry, source_ele=ele, rel=source_rel)
    for target_rel_id in ele.target_to_relationships:
        target_rel = registry.get(target_rel_id)
        if target_rel._metatype == "OwningMembership":
            target_side_resolve_owningmembership(registry=registry, target_ele=ele, rel=target_rel)
        elif target_rel._metatype == "FeatureMembership":
            target_side_resolve_featuremembership(registry=registry, target_ele=ele, rel=target_rel)
        elif target_rel._metatype == "FeatureTyping":
            target_side_resolve_featuretyping(registry=registry, target_ele=ele, rel=target_rel)

In [ ]:
test_registry2 = ModelRegistry()

# Pass 1 - get non-relationship element data
for ele_data in element_raw_data:
    new_ele = Element()
    load_primary_attributes(element=new_ele, data=ele_data)
    if is_relationship(ele_data):
        load_relationship_attributes(element=new_ele, data=ele_data)
    test_registry2.add(new_ele)
        
# Pass 2 - get relationship element data and connect others
for ele_data in element_raw_data:
    if is_relationship(ele_data):
        
        for source_ele in ele_data['source']:
            test_registry2.get(source_ele['@id']).source_to_relationships.add(ele_data['@id'])
        for target_ele in ele_data['target']:
            test_registry2.get(target_ele['@id']).target_to_relationships.add(ele_data['@id'])
            
        rel = test_registry2.get(ele_data['@id'])
        dereference_relationship_ends(rel=rel, registry=test_registry2)

In [ ]:
ele_to_derive = test_registry2.get('0ae12f1c-e915-40b7-90dd-af4a62bf9e81')

In [ ]:
hasattr(ele_to_derive, "ownedElement")

In [ ]:
for trial_item in trial_items:
    evaluate_derived_properties(ele=test_registry2.get(trial_item), registry=test_registry2)

In [ ]:
ele_to_derive.owner[0].name

In [ ]:
ele_to_derive.throughOwningMembership

In [ ]:
ele_to_derive.reverseOwningMembership

In [ ]:
[feature.name for feature in test_registry2.get_all_of_metatype("FeatureMembership")[2].ownedMemberFeature]

In [ ]:
example_ele_with_feature = [feature for feature in test_registry2.get_all_of_metatype("FeatureMembership")[2].owningRelatedElement][0]

In [ ]:
example_ele_with_feature.ownedElement["engineStarted"].name

In [ ]:
example_ele_with_feature.feature["engineStarted"].name

In [ ]:
example_ele_with_feature.feature["engineStarted"]

In [ ]:
[f"{feature.typedFeature[0].name}: {feature.type[0].name}"  for feature in test_registry2.get_all_of_metatype("FeatureTyping")]

In [ ]:
test_registry2.get_all_of_metatype("FeatureTyping")[4].typedFeature[0].type[0].name

In [ ]:
test_registry2.get_all_of_metatype("FeatureTyping")[4].typedFeature[0].name